In [2]:
import pandas as pd

orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [3]:
import json

with open("users.json") as f:
    users = pd.json_normalize(json.load(f))

users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [4]:
import sqlite3

conn = sqlite3.connect(":memory:")
with open("restaurants.sql") as f:
    conn.executescript(f.read())

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [5]:
merged = orders.merge(users, on="user_id", how="left")
merged = merged.merge(restaurants, on="restaurant_id", how="left")


In [6]:
merged.to_csv("Final_food_delivery_dataset.csv", index=False)

In [7]:
import pandas as pd

df = pd.read_csv("Final_food_delivery_dataset.csv")
df["order_date"] = pd.to_datetime(df["order_date"], dayfirst=True)

In [8]:
df["month"] = df["order_date"].dt.to_period("M")

df.groupby("month").agg(
    total_orders=("order_id","count"),
    total_revenue=("total_amount","sum")
).reset_index()

,month,total_orders,total_revenue
0,2023-01,804,645818.76
1,2023-02,785,630867.40
2,2023-03,903,716738.98
3,2023-04,812,666097.18
4,2023-05,844,668428.61
5,2023-06,784,610822.93
6,2023-07,859,688559.45
7,2023-08,851,694987.58
8,2023-09,812,653838.07
9,2023-10,863,699187.73


In [9]:
df.groupby("user_id").agg(
    total_orders=("order_id","count"),
    total_spent=("total_amount","sum"),
    avg_order_value=("total_amount","mean")
)

,total_orders,total_spent,avg_order_value
user_id,,,
1,1,1289.66,1289.660000
2,10,7564.12,756.412000
3,2,1839.51,919.755000
4,4,3741.16,935.290000
5,5,5742.88,1148.576000
...,...,...,...
2996,3,1533.54,511.180000
2997,5,5310.32,1062.064000
2998,6,4241.47,706.911667


In [10]:
df.groupby("city").agg(
    total_orders=("order_id","count"),
    total_revenue=("total_amount","sum"),
    avg_order_value=("total_amount","mean")
).sort_values("total_revenue", ascending=False)

,total_orders,total_revenue,avg_order_value
city,,,
Bangalore,2751,2206946.58,802.234308
Chennai,2469,1990513.03,806.202118
Pune,2430,1924797.93,792.097914
Hyderabad,2350,1889366.58,803.985779


In [11]:
df.groupby("cuisine").agg(
    total_orders=("order_id","count"),
    total_revenue=("total_amount","sum"),
    avg_order_value=("total_amount","mean")
).sort_values("total_revenue", ascending=False)

,total_orders,total_revenue,avg_order_value
cuisine,,,
Mexican,2581,2085503.09,808.021344
Italian,2532,2024203.80,799.448578
Indian,2469,1971412.58,798.466011
Chinese,2418,1930504.65,798.389020
